<a href="https://colab.research.google.com/github/shachi-i/Mini-RAG-Pipeline-for-Test-Case-Retrieval-Generation/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Test Case Retrieval for RAG-Assisted Generation**

loading the test cases


In [48]:
test_cases = [
  {
    "title": "Logged-in user completes checkout with valid credit card",
    "steps": [
      "Navigate to product page and add an in-stock item to the cart",
      "Proceed to checkout",
      "Confirm shipping address and choose standard delivery",
      "Select 'Credit Card' as payment method",
      "Enter valid credit card details",
      "Submit order"
    ],
    "expected_results": [
      "Order is successfully placed.",
      "Confirmation message is shown with order ID."
    ]
  },
  {
    "title": "Guest user attempts checkout without logging in",
    "steps": [
      "Add an in-stock item to the cart",
      "Click on checkout",
      "Try to proceed without logging in"
    ],
    "expected_results": [
      "User is redirected to login or sign-up page.",
      "Prompt appears asking to log in before proceeding."
    ]
  },
  {
    "title": "Checkout fails with expired credit card",
    "steps": [
      "Add item to cart",
      "Proceed to checkout",
      "Enter expired credit card information",
      "Submit payment"
    ],
    "expected_results": [
      "Transaction is declined.",
      "Error message indicates expired card."
    ]
  },
  {
    "title": "User uses saved address and saved card to checkout",
    "steps": [
      "Log in with user account",
      "Add item to cart",
      "Proceed to checkout",
      "Use pre-saved address",
      "Use pre-saved credit card",
      "Submit order"
    ],
    "expected_results": [
      "Order is placed successfully using saved information."
    ]
  },
  {
    "title": "User enters invalid CVV number",
    "steps": [
      "Log in and add item to cart",
      "Choose 'Credit Card' at payment step",
      "Enter invalid CVV number",
      "Try to complete the purchase"
    ],
    "expected_results": [
      "System displays error.",
      "Transaction is blocked."
    ]
  },
  {
    "title": "Server returns 500 error during final checkout submission",
    "steps": [
      "Add item to cart",
      "Proceed to checkout",
      "Enter valid details",
      "Submit the order",
      "Simulate server-side 500 error"
    ],
    "expected_results": [
      "Order submission fails.",
      "User-friendly error message is shown.",
      "Order is not processed."
    ]
  },
  {
    "title": "User tries to checkout with insufficient balance",
    "steps": [
      "Add item to cart",
      "Proceed to checkout",
      "Enter credit card with insufficient balance",
      "Submit payment"
    ],
    "expected_results": [
      "Transaction is declined.",
      "Error about insufficient funds."
    ]
  },
  {
    "title": "Form validation prevents checkout with missing shipping address",
    "steps": [
      "Add item to cart",
      "Proceed to checkout",
      "Leave shipping address blank",
      "Try to submit order"
    ],
    "expected_results": [
      "Validation error shown.",
      "Checkout does not proceed."
    ]
  },
  {
    "title": "Apply discount code during checkout",
    "steps": [
      "Add item to cart",
      "Proceed to checkout",
      "Enter valid discount code",
      "Submit order"
    ],
    "expected_results": [
      "Order is placed with discount applied."
    ]
  },
  {
    "title": "Invalid discount code is rejected at checkout",
    "steps": [
      "Add item to cart",
      "Proceed to checkout",
      "Enter invalid discount code",
      "Attempt to apply code"
    ],
    "expected_results": [
      "Error message shown.",
      "Discount not applied."
    ]
  }
]

In [49]:
!pip install -q sentence-transformers faiss-cpu

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import json
import faiss

In [50]:
model = SentenceTransformer("all-MiniLM-L6-v2")

making chunks and embedding

In [51]:
texts = []
for tc in test_cases:
    text = tc["title"] + ". " + " ".join(tc["steps"]) + " " + " ".join(tc["expected_results"])
    texts.append(text)

embeddings = model.encode(texts, convert_to_numpy=True)

In [52]:
user_prompt = "Check what happens when a logged-in user uses a saved card and the server fails."
prompt_embedding = model.encode([user_prompt])


using faiss for similarity search

In [53]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)
distances, indices = index.search(prompt_embedding, 3)

retrieving and printing top 3 relevant test cases

In [63]:
print("\nTop 3 Relevant Test Cases for Prompt:\n")
for i, idx in enumerate(indices[0]):
    tc = test_cases[idx]
    print(f"{i+1}. {tc['title']}")
    for step in tc["steps"]:
        print("   -", step)
    print(" Expected Results:")
    for res in tc["expected_results"]:
        print("     •", res)
    print("\n---\n")



Top 3 Relevant Test Cases for Prompt:

1. User uses saved address and saved card to checkout
   - Log in with user account
   - Add item to cart
   - Proceed to checkout
   - Use pre-saved address
   - Use pre-saved credit card
   - Submit order
 Expected Results:
     • Order is placed successfully using saved information.

---

2. Guest user attempts checkout without logging in
   - Add an in-stock item to the cart
   - Click on checkout
   - Try to proceed without logging in
 Expected Results:
     • User is redirected to login or sign-up page.
     • Prompt appears asking to log in before proceeding.

---

3. User enters invalid CVV number
   - Log in and add item to cart
   - Choose 'Credit Card' at payment step
   - Enter invalid CVV number
   - Try to complete the purchase
 Expected Results:
     • System displays error.
     • Transaction is blocked.

---

